In [43]:
import cv2 
import numpy as np 
import os
import pandas as pd 
import torch

from importlib import import_module
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.transforms.functional import hflip, vflip 
from tqdm import tqdm 

from skp.toolbox.functions import load_model_from_config

In [41]:
class MelanomaDataset(Dataset):

    def __init__(self, files, image_sizes):
        self.files = files
        self.image_sizes = image_sizes
        self.transforms = {image_size: v2.Compose([v2.ToImage(), v2.Resize(image_size)]) for image_size in image_sizes}
        self.center_crop = {image_size: v2.CenterCrop((image_size, image_size)) for image_size in image_sizes}

    def __len__(self):
        return len(self.files)
    
    def preprocess(self, img, image_size):
        img = self.transforms[image_size](img.copy())
        h, w = img.shape[1:]
        if h > w:
            crop0 = img[:, :image_size]
            crop1 = img[:, -image_size:]
        elif h < w:
            crop0 = img[:, :, :image_size]
            crop1 = img[:, :, -image_size:]
        elif h == w:
            crop0 = hflip(img)
            crop1 = vflip(img)
        crop2 = self.center_crop[image_size](img)
        return torch.stack([crop0, crop1, crop2]).float()
    
    def __getitem__(self, i):
        img = cv2.imread(self.files[i], cv2.IMREAD_COLOR)
        img_dict = {image_size: self.preprocess(img, image_size) for image_size in self.image_sizes}
        return img_dict

In [18]:
cfg_name = "melanoma.cfg_2019_baseline"
cfg_512 = import_module(f"skp.configs.{cfg_name}").cfg
cfg_name = "melanoma.cfg_2019_baseline_640"
cfg_640 = import_module(f"skp.configs.{cfg_name}").cfg
cfg_name = "melanoma.cfg_2019_baseline_768"
cfg_768 = import_module(f"skp.configs.{cfg_name}").cfg

In [27]:
model_dict = {}
model_dict[512] = load_model_from_config(cfg_512, weights_path="/home/ian/projects/SKP/experiments/melanoma/melanoma.cfg_2019_baseline/e1638ae8/fold0/checkpoints/last.ckpt", device="cuda", eval_mode=True)
model_dict[640] = load_model_from_config(cfg_640, weights_path="/home/ian/projects/SKP/experiments/melanoma/melanoma.cfg_2019_baseline_640/8bfd2448/fold1/checkpoints/last.ckpt", device="cuda", eval_mode=True)
model_dict[768] = load_model_from_config(cfg_768, weights_path="/home/ian/projects/SKP/experiments/melanoma/melanoma.cfg_2019_baseline_768/c9739c02/fold2/checkpoints/last.ckpt", device="cuda", eval_mode=True)

In [ ]:
df = pd.read_csv("/mnt/stor/datasets/ISIC/2020/ISIC_2020_Training_GroundTruth_v2.csv")
files = df["image_name"].tolist()
files = [f"/mnt/stor/datasets/ISIC/2020/train/{f}.jpg" for f in files]
files[:2]

In [ ]:
dataset = MelanomaDataset(files, image_sizes=[512, 640, 768])
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

pred_list = []
for batch in tqdm(dataloader):
    y_list = []
    for k, v in batch.items():
        x = v[0].cuda()
        with torch.inference_mode():
            y = model_dict[k]({"x": x})
            y = y["logits"].softmax(dim=1).mean(0)
        y_list.append(y)
    y = torch.stack(y_list).mean(0)
    pred_list.append(y.cpu().numpy())

In [ ]:
pred_df = pd.DataFrame(np.stack(pred_list))
pred_df.columns = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC", "UNK"]
pred_df["filepath"] = files[:len(pred_df)]
pred_df["filepath"] = pred_df.filepath.apply(lambda x: os.path.basename(x))
pred_df

In [51]:
pred_df.to_csv("/mnt/stor/datasets/ISIC/2020/train_pseudolabels_from_2019_models.csv", index=False)

In [52]:
gt_df = pd.read_csv("/mnt/stor/datasets/ISIC/2020/ISIC_2020_Training_GroundTruth.csv")
pred_df["image_name"] = pred_df.filepath.apply(lambda x: x.split(".")[0])
gt_df = gt_df.merge(pred_df, on="image_name")

In [ ]:
from sklearn.metrics import roc_auc_score


roc_auc_score(gt_df.target.values, gt_df.MEL.values)

: 

In [ ]:
import matplotlib.pyplot as plt

for k, v in batch.items():
    imgs = v[0]
    for img in imgs:
        plt.imshow(img.permute(1, 2, 0).long().numpy())
        plt.show()